In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColorFiltering") \
    .config("spark.master", "local") \
    .getOrCreate()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/adamlee/Library/Python/3.9/lib/python/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
colors = ["red", "blue", "green", "yellow", "orange", "purple", "black", "white", "grey", "pink"]


In [ ]:
df = spark.read.parquet("/path/to/your/parquet/files/*")


In [ ]:
from pyspark.sql.functions import col, lower

# Dynamically construct a filter condition for all colors
filter_condition = None

for color in colors:
    current_condition = lower(col("TEXT")).contains(color)
    if filter_condition is None:
        filter_condition = current_condition
    else:
        filter_condition = filter_condition | current_condition

# Apply the filter condition to the DataFrame
filtered_df = df.filter(filter_condition)


In [ ]:
filtered_df.show()


In [ ]:
import pandas as pd
import os

directory = '/path/to/your/parquet/files/'
filtered_frames = []  # A list to hold the filtered chunks

for filename in os.listdir(directory):
    if filename.endswith(".snappy.parquet"):
        filepath = os.path.join(directory, filename)
        for chunk in pd.read_parquet(filepath, chunksize=10000):  # Adjust chunksize based on your memory constraints
            filtered_chunk = chunk[chunk['TEXT'].apply(contains_color)]
            filtered_frames.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_frames)

# Display some of the filtered DataFrame to verify
print(filtered_df.head())
